# Reflection


In the context of LLM agent building, reflection refers to the process of prompting an LLM to observe its past steps (along with potential observations from tools/the environment) to assess the quality of the chosen actions.
This is then used downstream for things like re-planning, search, or evaluation.

![Reflection](attachment:fc393f72-3401-4b86-b0d3-e4789b640a27.png)

This notebook demonstrates a very simple form of reflection in LangGraph.

## Setup

First, let's install our required packages and set our API keys

In [1]:
%pip install -U --quiet  langgraph langchain-fireworks
%pip install -U --quiet tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.6 MB/s eta 0:00:00


In [2]:
import getpass
import os


def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var)


_set_if_undefined("TAVILY_API_KEY")
_set_if_undefined("FIREWORKS_API_KEY")

TAVILY_API_KEY··········
FIREWORKS_API_KEY··········


<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>.
    </p>
</div>

## Generate

For our example, we will create a "5 paragraph essay" generator. First, create the generator:


In [6]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_fireworks import ChatFireworks

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatFireworks(
    model="accounts/fireworks/models/llama4-maverick-instruct-basic", max_tokens=32768
)
generate = prompt | llm

In [7]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

The Little Prince, a novella written by Antoine de Saint-Exupéry, has been a beloved tale among children and adults alike since its publication in 1943. Despite being written over seven decades ago, the story remains remarkably relevant in modern childhood. One of the primary reasons for its enduring relevance is its exploration of complex themes such as friendship, love, and the importance of human connections. The Little Prince's poignant portrayal of the protagonist's journey and his interactions with various characters serves as a powerful reminder to children of the value of nurturing meaningful relationships in their own lives.

In today's digital age, children are increasingly surrounded by technology and social media, which can often create a false sense of connection. The Little Prince's emphasis on the importance of genuine, face-to-face relationships is a valuable lesson for children growing up in this environment. The prince's experiences with the rose, in particular, illus

### Reflect

In [8]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [9]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

**Critique and Recommendations**

The essay effectively explores the relevance of The Little Prince in modern childhood, highlighting its timeless themes and messages. Here's a detailed critique and recommendations for improvement:

**Strengths:**

1. The essay clearly articulates the novella's enduring relevance and its significance in modern childhood.
2. The writer provides specific examples from the novella to support their arguments, demonstrating a good understanding of the text.
3. The essay is well-structured and easy to follow, with a logical flow of ideas.

**Weaknesses:**

1. The essay is somewhat superficial in its analysis, and the writer could benefit from delving deeper into the themes and symbolism of The Little Prince.
2. The writing is clear but lacks a distinctive voice and style, making it somewhat generic.
3. Some paragraphs feel a bit formulaic, with a straightforward topic sentence followed by supporting evidence.

**Recommendations:**

1. **Depth of Analysis:** 

### Repeat

And... that's all there is too it! You can repeat in a loop for a fixed number of steps, or use an LLM (or other check) to decide when the finished product is good enough.

In [10]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

The Little Prince, a novella written by Antoine de Saint-Exupéry, has been a beloved tale among children and adults alike since its publication in 1943. Despite being written over seven decades ago, the story remains remarkably relevant in modern childhood, its themes and messages continuing to resonate with young readers. One of the primary reasons for its enduring relevance is its poignant and piercing exploration of the human experience, delving into complex themes such as friendship, love, and the importance of human connections. The Little Prince's journey, marked by his encounters with various characters, serves as a powerful allegory for the challenges and complexities of growing up, and the significance of nurturing meaningful relationships in one's life.

In today's digital age, children are increasingly surrounded by technology and social media, which can often create a false sense of connection. The Little Prince's emphasis on the importance of genuine, face-to-face relation

## Define graph

Now that we've shown each step in isolation, we can wire it up in a graph.

In [11]:
from typing import Annotated, List, Sequence
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


async def generation_node(state: State) -> State:
    return {"messages": [await generate.ainvoke(state["messages"])]}


async def reflection_node(state: State) -> State:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]
    res = await reflect.ainvoke(translated)
    # We treat the output of this as human feedback for the generator
    return {"messages": [HumanMessage(content=res.content)]}


builder = StateGraph(State)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_edge(START, "generate")


def should_continue(state: State):
    if len(state["messages"]) > 6:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [12]:
config = {"configurable": {"thread_id": "1"}}

In [13]:
async for event in graph.astream(
    {
        "messages": [
            HumanMessage(
                content="Generate an essay on the topicality of The Little Prince and its message in modern life"
            )
        ],
    },
    config,
):
    print(event)
    print("---")

{'generate': {'messages': [AIMessage(content='The novella "The Little Prince" by Antoine de Saint-Exupéry has been a timeless classic since its publication in 1943. The story follows the journey of a young prince as he travels from his own planet to Earth, learning valuable lessons about friendship, love, and what truly matters in life. Despite being written over 75 years ago, "The Little Prince" remains remarkably topical, and its message continues to resonate with readers of all ages in modern life. One of the primary reasons for its enduring relevance is its thoughtful exploration of the human condition, which transcends time and cultural boundaries.\n\nAt its core, "The Little Prince" is a poignant critique of modern society\'s values and priorities. The little prince\'s encounters with various characters on his journey serve as a commentary on the superficiality and materialism that can pervade human relationships. The prince\'s interactions with the businessman, who is obsessed w

In [ ]:
state = graph.get_state(config)

In [ ]:
ChatPromptTemplate.from_messages(state.values["messages"]).pretty_print()

================================ Human Message =================================

Generate an essay on the topicality of The Little Prince and its message in modern life

================================== Ai Message ==================================

Title: The Little Prince: A Topical Allegory for Modern Life

Introduction:
Antoine de Saint-Exupéry's "The Little Prince" is a classic novella that has captured the hearts of millions since its publication in 1943. While it might be easy to dismiss this work as a children's story, its profound themes and timeless message make it a relevant and topical piece in modern life. This essay will explore the allegorical nature of "The Little Prince" and discuss how its message can be applied to the complexities of the modern world.

Body Paragraph 1 - The Allegory of the Little Prince:
"The Little Prince" is an allegorical tale that explores various aspects of the human condition through its whimsical characters and situations. The Little Princ

## Conclusion

Now that you've applied reflection to an LLM agent, I'll note one thing: self-reflection is inherently cyclic: it is much more effective if the reflection step has additional context or feedback (from tool observations, checks, etc.). If, like in the scenario above, the reflection step simply prompts the LLM to reflect on its output, it can still benefit the output quality (since the LLM then has multiple "shots" at getting a good output), but it's less guaranteed.
